In [1]:
import random
import json
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

from utils.util import *

In [2]:
def get_pred_record(pred_triggers):
    pred_record = {'argument':{}}
    try:
        for pred_arg in pred_triggers:
            if 'tri counter' in pred_arg[-1].keys():
                pred_record['trigger'] = pred_arg[0]
                pred_record['event'] = pred_arg[1]
                continue
            role = pred_arg[1]
            arg = pred_arg[0]
            if not pred_record['argument'].get(role):
                pred_record['argument'][role] = []
            pred_record['argument'][role].append(arg)
    except:
        pass
    return pred_record
    

In [3]:
def get_gold_record(gold_triggers, gold_info):
    gold_record = {'argument':{}}
    try:
        event_type = gold_info[1]
        trigger = gold_triggers[-1][-1]
        gold_record['event'] = event_type
        gold_record['trigger'] = trigger
        for i in gold_info[0]:
            if i['event type'] == event_type:
                for role, args in i['arguments'].items():
                    if not gold_record['argument'].get(role):
                        gold_record['argument'][role] = []
                    for arg in args:
                        gold_record['argument'][role].append(arg['argument text'])
    except:
        pass
    return gold_record

In [4]:
def safe_div(a, b):
    if b != 0:
        return a/b
    else:
        return 0

def eval_skip_span(pred_record, gold_recod):
    try:
        arg_c = 0
        if gold_record['event'] == pred_record['event']:
            for role, args in gold_record['argument'].items():
                if not pred_record['argument'].get(role):
                    continue
                for arg in args:
                    if arg.lower() in [i.lower() for i in pred_record['argument'][role]]:
                        arg_c += 1
        
        arg_i = 0
        
        gold_arg = []
        pred_arg = []
        for role, args in gold_record['argument'].items():
            gold_arg.extend(args)
        for role, args in pred_record['argument'].items():
            pred_arg.extend(args)
        
        gold_num = len(gold_arg)
        pred_num = len(pred_arg)
        
        if gold_record['event'] == pred_record['event']:
            for arg in pred_arg:
                if arg.lower() in [i.lower() for i in gold_arg]:
                    arg_i += 1
        
        prec_arg_i = safe_div(arg_i, pred_num)
        recall_arg_i = safe_div(arg_i, gold_num)
        f1_arg_i = 2 * safe_div(prec_arg_i*recall_arg_i, prec_arg_i + recall_arg_i)
    
        prec_arg_c = safe_div(arg_c, pred_num)
        recall_arg_c = safe_div(arg_c, gold_num)
        f1_arg_c = 2 * safe_div(prec_arg_c*recall_arg_c, prec_arg_c + recall_arg_c)
    
        return prec_arg_i, recall_arg_i, f1_arg_i, prec_arg_c, recall_arg_c, f1_arg_c
    except:
        return 0,0,0,0,0,0

In [7]:
dev = load_json('./exp_data/pred.dev.json')

In [29]:
y = []
for i in dev:
    pred_triggers = i['pred triggers']
    gold_triggers = i['gold triggers']
    gold_info = i['gold info']
    if gold_triggers:
        y.append(i)

In [89]:
i = y[7]
pred_triggers = i['pred triggers']
gold_triggers = i['gold triggers']
gold_info = i['gold info']

In [91]:
pred_triggers

[['summit', 'Contact:Meet', {'tri counter': 0}],
 ['Putin', 'Entity', {'cor tri cnt': 0}],
 ['Germany', 'Place', {'cor tri cnt': 0}],
 ['France', 'Place', {'cor tri cnt': 0}]]

In [93]:
gold_triggers

[[5, 6], [8, 9, 'Entity'], [2, 4, 'Entity']]

In [95]:
gold_info[0]

[{'trigger text': 'summit',
  'trigger span': [5, 6],
  'event type': 'Contact:Meet',
  'arguments': {'Entity': [{'argument text': 'Vladimir Putin',
     'argument span': [2, 4]},
    {'argument text': 'leaders', 'argument span': [8, 9]}]},
  'passage': 'Russian President Vladimir Putin \'s summit with the leaders of Germany and France may have been a failure that proves there can be no long - term " peace camp " alliance following the end of war in Iraq , government sources were quoted as saying at the weekend .',
  'tokens': ['Russian',
   'President',
   'Vladimir',
   'Putin',
   "'s",
   'summit',
   'with',
   'the',
   'leaders',
   'of',
   'Germany',
   'and',
   'France',
   'may',
   'have',
   'been',
   'a',
   'failure',
   'that',
   'proves',
   'there',
   'can',
   'be',
   'no',
   'long',
   '-',
   'term',
   '"',
   'peace',
   'camp',
   '"',
   'alliance',
   'following',
   'the',
   'end',
   'of',
   'war',
   'in',
   'Iraq',
   ',',
   'government',
   'sou

In [97]:
s = "Washington, DC - The <Plaintiff>Bechtel Corporation</Plaintiff> and a <Plaintiff>buyer</Plaintiff> filed a <SUE><trigger>suit</trigger></SUE> against <Defendant>bolivia</Defendant> in <Place>Washington</Place>, with the <Adjudicator>Court</Adjudicator> overseeing the proceedings. Meanwhile, a <TRIAL-HEARING><trigger>hearing</trigger></TRIAL-HEARING> took place at <Place>Capitol Hill</Place>, where <Prosecutor>garcia</Prosecutor> prosecuted <Defendant>bolivia</Defendant> before <Adjudicator>Judge John S. Martin</Adjudicator>. Separately, the <Adjudicator>new york state tax commission</Adjudicator> imposed a <FINE><trigger>fine</trigger></FINE> on <Entity>bolivia</Entity> at the <Place>La Plata County Court</Place>."

In [99]:
re.sub('<.*?>', '', s)

'Washington, DC - The Bechtel Corporation and a buyer filed a suit against bolivia in Washington, with the Court overseeing the proceedings. Meanwhile, a hearing took place at Capitol Hill, where garcia prosecuted bolivia before Judge John S. Martin. Separately, the new york state tax commission imposed a fine on bolivia at the La Plata County Court.'

In [101]:
s = "Washington - The <Plaintiff>Bechtel Corporation</Plaintiff> and <Plaintiff>buyer</Plaintiff> filed a <SUE><trigger>suit</trigger></SUE> against <Defendant>bolivia</Defendant> in <Place>Washington</Place>, where the <Adjudicator>Court</Adjudicator> will hear the case. Meanwhile, at <Place>Capitol Hill</Place>, a <TRIAL-HEARING><trigger>hearing</trigger></TRIAL-HEARING> presided over by <Adjudicator>Judge John S. Martin</Adjudicator> saw <Prosecutor>garcia</Prosecutor> prosecuting <Defendant>bolivia</Defendant>. Separately, the <Adjudicator>new york state tax commission</Adjudicator> imposed a <FINE><trigger>fine</trigger></FINE> on <Entity>bolivia</Entity> at the <Place>La Plata County Court</Place>."

In [103]:
re.sub('<.*?>', '', s)

'Washington - The Bechtel Corporation and buyer filed a suit against bolivia in Washington, where the Court will hear the case. Meanwhile, at Capitol Hill, a hearing presided over by Judge John S. Martin saw garcia prosecuting bolivia. Separately, the new york state tax commission imposed a fine on bolivia at the La Plata County Court.'

In [63]:
total_score = np.array([0, 0, 0, 0, 0, 0])
zero = 0
for i in dev:
    pred_triggers = i['pred triggers']
    gold_triggers = i['gold triggers']
    gold_info = i['gold info']
    
    pred_record = get_pred_record(pred_triggers)
    gold_record = get_gold_record(gold_triggers, gold_info)
    
    if not pred_record.get('event') and not gold_record.get('event'):
        zero += 1
        continue
    if not pred_record['argument'] and not gold_record['argument']:
        zero += 1
        continue
    score = np.array(eval_skip_span(pred_record, gold_record))

    total_score = total_score + score

print(total_score / (len(dev) - zero))

[0.23081807 0.22423077 0.21702483 0.22333944 0.21418803 0.20880749]


In [51]:
gold_triggers

[]

In [151]:
for i, d in enumerate(dev):
    if d['gold triggers']:
        if i % 50 == 0:
            print(d['gold text'])
            print(d['pred text'])
            print()
            

Event trigger is war 
 some attacker attacked some facility, someone, or some organization by some way in Lebanon.
Event trigger is <Trigger>

Event trigger is meeting 
 Pyongyang met at somewhere.
Event trigger is <Trigger>

Event trigger is launched 
 founder launched chain in somewhere.
Event trigger is launched 
 somebody or some organization launched some organization in somewhere.

Event trigger is demonstration 
 some people or some organization protest at somewhere.
Event trigger is <Trigger>

Event trigger is elections 
 somebody was elected a position, and the election was voted by some people or some organization in somewhere.
Event trigger is <Trigger>

Event trigger is met 
 Bush and Gerhard Schroeder met at Saint Petersburg.
Event trigger is met 
 Bush met at Saint Petersburg.

Event trigger is war 
 some attacker attacked some facility, someone, or some organization by some way in somewhere.
Event trigger is <Trigger>

Event trigger is repaying 
 Vivendi Universal paid s

In [3]:
train_all = train['all']

In [44]:
trigger = {}
argument = {}
for data in train_all:
    for rec in data[0]:
        e_type = rec['event type'].split(':')[-1].upper()
        if not trigger.get(e_type):
            trigger[e_type] = []
        if not argument.get(e_type):
            argument[e_type] = {}
            
        if not rec['trigger text'] in trigger[e_type]:
            trigger[e_type].append(rec['trigger text'])
        
        for role, v in rec['arguments'].items():
            if not argument[e_type].get(role):
                argument[e_type][role] = []
            argument[e_type][role].extend([arg['argument text'] for arg in v])

In [45]:
for i, (k,v) in enumerate(trigger.items()):
    trigger[k] = Series(v).drop_duplicates(ignore_index=True).to_list()

In [47]:
for i, (k,v) in enumerate(argument.items()):
    for role, args in v.items():
        argument[k][role] = Series(args).drop_duplicates(ignore_index=True).to_list()

In [49]:
save_json(trigger, './trigger_pool_ori.json')
save_json(argument, './argument_pool_ori.json')

In [33]:
a = DataFrame({'name':argument['Movement:Transport']['Vehicle']})

In [34]:
name_counts = a['name'].value_counts()

In [36]:
name_counts

name
boat                 1
convoy               1
ashters              1
trailers             1
ships                1
ship                 1
cars                 1
USS Constellation    1
bus                  1
trains               1
rocket               1
spacecraft           1
vehicle              1
van                  1
Sensation            1
underground          1
train                1
carriers             1
truck                1
rovers               1
flight               1
mars polar lander    1
soyuz                1
trucks               1
helicopter           1
vehicles             1
plane                1
ambulance            1
jet                  1
capsule              1
car                  1
mars express         1
tanks                1
air force one        1
aircraft             1
planes               1
flight 903           1
carrier              1
uss constellation    1
airliners            1
Name: count, dtype: int64

In [71]:
# to do 10 个类别
# 增强实验

gen_data = load_json('./data_sen_3_4_8_38.json')

In [72]:
trigger = []
arg = []
for k, v in gen_data.items():
    for p in v:
        for ek, ev in p['record'].items():
            trigger.append(ev['trigger'])
            for ak, ak in ev['argument'].items():
                arg.extend(ak)

In [62]:
train = []
with open('./train.w1.oneie.json', 'r') as f:
    for reco in f.readlines():
        train.append(json.loads(reco))

mulit_event = {}
event_data = {}
c_m = 0
for data in train:
    if len(data['event_mentions']) > 1:
        c_m += 1
        e_type = e['event_type']
        if not mulit_event.get(e_type):
            mulit_event[e_type] = []
        mulit_event[e_type].append(data)
    for e in data['event_mentions']:
        e_type = e['event_type']
        if not event_data.get(e_type):
            event_data[e_type] = []
        event_data[e_type].append(data)
test_model_data = {}

type_list = []
for k, v in event_data.items():
    if len(v) > 50:
        type_list.append(k)
    if len(type_list) > 9:
        break

In [65]:
train = []
with open('./test.w1.oneie.json', 'r') as f:
    for reco in f.readlines():
        train.append(json.loads(reco))

mulit_event = {}
event_data = {}
c_m = 0
for data in train:
    if len(data['event_mentions']) > 1:
        c_m += 1
        e_type = e['event_type']
        if not mulit_event.get(e_type):
            mulit_event[e_type] = []
        mulit_event[e_type].append(data)
    for e in data['event_mentions']:
        e_type = e['event_type']
        if not event_data.get(e_type):
            event_data[e_type] = []
        event_data[e_type].append(data)
test_model_data = {}

for k in type_list:
    # print('{}\t\t{}\t'.format(k, len(v)))
    try:
        test_model_data[k] = event_data[k]
    except:
        pass

with open('./test.w2.oneie.json', 'w') as f:
    for k, v in test_model_data.items():
        for rec in v:
            f.writelines(json.dumps(rec)+'\n')

In [68]:
print([i.split(':')[-1].upper() for i in type_list])

['TRANSPORT', 'ELECT', 'START-POSITION', 'ATTACK', 'END-POSITION', 'MEET', 'MARRY', 'PHONE-WRITE', 'TRANSFER-MONEY', 'SUE']


In [57]:
train = []
with open('./train.w2.oneie.json', 'r') as f:
    for reco in f.readlines():
        train.append(json.loads(reco))

In [59]:
train[0]

{'doc_id': 'CNN_CF_20030303.1900.00',
 'wnd_id': 'CNN_CF_20030303.1900.00-6',
 'entity_mentions': [{'id': 'CNN_CF_20030303.1900.00-6-E0',
   'start': 3,
   'end': 4,
   'entity_type': 'PER',
   'mention_type': 'UNK',
   'text': 'secretary'},
  {'id': 'CNN_CF_20030303.1900.00-6-E1',
   'start': 5,
   'end': 7,
   'entity_type': 'ORG',
   'mention_type': 'UNK',
   'text': 'homeland security'},
  {'id': 'CNN_CF_20030303.1900.00-6-E2',
   'start': 10,
   'end': 11,
   'entity_type': 'PER',
   'mention_type': 'UNK',
   'text': 'people'},
  {'id': 'CNN_CF_20030303.1900.00-6-E3',
   'start': 19,
   'end': 20,
   'entity_type': 'GPE',
   'mention_type': 'UNK',
   'text': 'Cuban'},
  {'id': 'CNN_CF_20030303.1900.00-6-E4',
   'start': 21,
   'end': 22,
   'entity_type': 'VEH',
   'mention_type': 'UNK',
   'text': 'boat'},
  {'id': 'CNN_CF_20030303.1900.00-6-E5',
   'start': 26,
   'end': 27,
   'entity_type': 'PER',
   'mention_type': 'UNK',
   'text': 'men'},
  {'id': 'CNN_CF_20030303.1900.00-6

In [55]:
mulit_event = {}
event_data = {}
c_m = 0
for data in train:
    if len(data['event_mentions']) > 1:
        c_m += 1
        e_type = e['event_type']
        if not mulit_event.get(e_type):
            mulit_event[e_type] = []
        mulit_event[e_type].append(data)
    for e in data['event_mentions']:
        e_type = e['event_type']
        if not event_data.get(e_type):
            event_data[e_type] = []
        event_data[e_type].append(data)
test_model_data = {}

for k, v in event_data.items():
    # print('{}\t\t{}\t'.format(k, len(v)))
    if len(v) > 50:
        test_model_data[k] = random.sample(v, 50)
    else:
        test_model_data[k] = v

In [56]:
for k, v in test_model_data.items():
    # print('{}\t\t{}\t'.format(k, len(v)))
    for i, rec in enumerate(v):
        if i < 5:
            print(json.dumps(rec))
        break
    break

Movement:Transport		50	
{"doc_id": "fsh_29774", "wnd_id": "fsh_29774-90", "entity_mentions": [{"id": "fsh_29774-90-E0", "start": 1, "end": 2, "entity_type": "PER", "mention_type": "UNK", "text": "son"}, {"id": "fsh_29774-90-E1", "start": 8, "end": 9, "entity_type": "FAC", "mention_type": "UNK", "text": "home"}], "relation_mentions": [], "event_mentions": [{"event_type": "Movement:Transport", "id": "fsh_29774-90-EV0", "trigger": {"start": 6, "end": 7, "text": "moving"}, "arguments": [{"entity_id": "fsh_29774-90-E0", "text": "son", "role": "Artifact"}, {"entity_id": "fsh_29774-90-E1", "text": "home", "role": "Destination"}]}], "entity_coreference": [], "event_coreference": [], "tokens": ["My", "son", "'s", "going", "to", "be", "moving", "back", "home", "next", "year", ",", "and", "he", "'s", "a", "joy", "."], "pieces": ["My", "son", "'s", "going", "to", "be", "moving", "back", "home", "next", "year", ",", "and", "he", "'s", "a", "joy", "."], "token_lens": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [63]:
with open('./train.w2.oneie.json', 'w') as f:
    for k, v in test_model_data.items():
        for rec in v:
            f.writelines(json.dumps(rec)+'\n')